In [18]:
from tqdm import tqdm
import pandas as pd
from binance.client import Client

import numpy as np

In [2]:
client = Client()

In [3]:
coins = ('BTCUSDT','ETHUSDT','BNBUSDT','SOLUSDT','ADAUSDT','XRPUSDT','DOTUSDT','LUNAUSDT',
  'DOGEUSDT','AVAXUSDT','SHIBUSDT','MATICUSDT','LTCUSDT','UNIUSDT','ALGOUSDT','TRXUSDT',
         'LINKUSDT','MANAUSDT','ATOMUSDT','VETUSDT')

In [4]:
def getminutedata(symbol,lookback):
    frame = pd.DataFrame(client.get_historical_klines(symbol,
                                                      '1m',
                                                     lookback + ' days ago UTC'))
    frame = frame.iloc[:,:5]
    frame.columns = ['Time','Open','High','Low','Close']
    frame[['Open','High','Low','Close']] = frame[['Open','High','Low','Close']].astype(float)
    frame.Time = pd.to_datetime(frame.Time, unit='ms')
    return frame

In [5]:
# getminutedata('BTCUSDT','1')

In [6]:
from sqlalchemy import create_engine

In [7]:
engine = create_engine('sqlite:///Cryptoprices.db')

In [9]:
for coin in tqdm(coins):
    getminutedata(coin, '30').to_sql(coin, engine,if_exists='append', index=False)

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [08:16<00:00, 24.82s/it]


In [20]:
test = pd.read_sql('BTCUSDT',engine).set_index('Time')

In [19]:
# test

,Open,High,Low,Close
Time,,,,
2022-03-28 00:25:00,46713.19,46761.28,46711.00,46761.27
2022-03-28 00:26:00,46761.28,46776.23,46761.27,46773.42
2022-03-28 00:27:00,46773.43,46777.20,46765.78,46777.19
2022-03-28 00:28:00,46777.19,46798.99,46777.19,46785.26
2022-03-28 00:29:00,46785.27,46820.33,46785.26,46817.92
...,...,...,...,...
2022-04-27 00:55:00,38170.00,38206.16,38166.17,38166.17
2022-04-27 00:56:00,38166.18,38178.33,38140.00,38146.38
2022-04-27 00:57:00,38146.38,38184.67,38115.08,38158.47


In [45]:
def technicals(df):
    df = df.copy()
    df = df.resample('15min').ffill()
    df.dropna(inplace=True)
    df['return'] = np.log(df.Close.pct_change()+1)
    df['SMA_fast'] = df.Close.rolling(7).mean()
    df['SMA_slow'] = df.Close.rolling(25).mean()
    df['position'] = np.where(df['SMA_fast'] > df['SMA_slow'], 1,0)
    df['strategyreturn'] = df['position'].shift(1) * df['return']
    df.dropna(inplace=True)
    return df

In [46]:
technicals(test)

ValueError: cannot reindex a non-unique index with a method or limit

In [31]:
np.exp(technicals(test)[['return','strategyreturn']].sum())-1

return           -0.186596
strategyreturn   -0.392396
dtype: float64

In [41]:
for coin in coins:
    df = pd.read_sql(coin,engine).set_index('Time')
    print(coin)
    trades = technicals(df).position.diff().value_counts().iloc[1:].sum()
    costs = trades * 0.00075
    print(np.exp(technicals(df)['return'].sum())-1),
    print(np.exp(technicals(df)['strategyreturn'].sum())-1-costs)

BTCUSDT
-0.1865963286055452
-3.763645984890501
ETHUSDT
-0.14428817529691906
-3.850027285456746
BNBUSDT
-0.10081490104777935
-3.669619475196663
SOLUSDT
-0.09539535761468687
-3.7850103593966247
ADAUSDT
-0.2965957446808788
-4.206713904226542
XRPUSDT
-0.2507552870090841
-3.2347489237129294
DOTUSDT
-0.25978647686834244
-1.8639340941111309
LUNAUSDT
-0.05058158147477221
-1.9196511311840907
DOGEUSDT
-0.056202878683967006
-1.9344422868606
AVAXUSDT
-0.22766505997086195
-1.9215502779971128
SHIBUSDT
-0.11128165771297416
-1.8186956438328623
MATICUSDT
-0.264028352037742
-1.9966757540385336
LTCUSDT
-0.24036979969181693
-2.33712833412761
UNIUSDT
-0.2921052631578863
-2.1588439563240884
ALGOUSDT
-0.31488070322310324
-1.8137205208251
TRXUSDT
-0.10118702953095038
-1.6419430642661816
LINKUSDT
-0.25746714456388686
-1.975605500589886
MANAUSDT
-0.3313404192500716
-2.01938549993744
ATOMUSDT
-0.3303776683086944
-1.9443610139116405
VETUSDT
-0.3179336972440251
-1.6174368567118673


In [34]:
technicals(test).tail(5)

,Open,High,Low,Close,return,SMA_fast,SMA_slow,position,strategyreturn
Time,,,,,,,,,
2022-04-27 00:55:00,38170.00,38206.16,38166.17,38166.17,-0.000100,38167.751429,38109.7960,1,-0.000100
2022-04-27 00:56:00,38166.18,38178.33,38140.00,38146.38,-0.000519,38165.952857,38107.5564,1,-0.000519
2022-04-27 00:57:00,38146.38,38184.67,38115.08,38158.47,0.000317,38158.984286,38106.4188,1,0.000317
2022-04-27 00:58:00,38158.47,38168.47,38130.25,38168.46,0.000262,38158.310000,38106.2632,1,0.000262
2022-04-27 00:59:00,38168.47,38170.00,38157.86,38168.04,-0.000011,38158.348571,38106.3536,1,-0.000011


In [38]:
technicals(test).position.diff().value_counts().iloc[1:].sum()*0.00075

3.37125